# Example-generator.ipynb

Test area to construct and example tilt out of the tilt-schema.

Will be turned into .py file that can be executed as a GitHub action. 

In [ ]:
import os 
import json
import glob
import jsonschema
from jsonschema import Draft7Validator

In [10]:
import json
import jsonschema

# Load JSON schema
with open('combined.json', 'r') as schema_file:
    schema_data = json.load(schema_file)

# Build empty JSON based on schema
empty_json = {}
for key, value in schema_data.get('properties', {}).items():
    if value.get('type') == 'object':
        empty_json[key] = {}
    elif value.get('type') == 'array':
        empty_json[key] = []
    else:
        empty_json[key] = None

# Fill empty JSON with examples provided in schema
def update_json_with_example(json_obj, schema_obj):
    for key, value in schema_obj.get('properties', {}).items():


        if value.get('type') == 'object':
            json_obj[key] = {}
            update_json_with_example(json_obj[key], value)
        elif value.get('type') == 'array':
            json_obj[key] = []
            if value.get('items', {}).get('type') == 'object':
                json_obj[key].append({})
                update_json_with_example(json_obj[key][0], value['items'])
        else:
            example = None
            if 'anyOf' in value:
                for subschema in value['anyOf']:
                    try:
                        jsonschema.validate({key: subschema['examples'][0]}, {key: subschema})
                        example = subschema['examples'][0]
                        break
                    except jsonschema.ValidationError:
                        pass
            else:
                # Check if field is optional
                if value.get('default') is not None or value.get('nullable') or value.get('optional'):
                    # If optional, use default value or set to null
                    json_obj[key] = value.get('default', None)
                else:
                    # Otherwise, use example value
                    example = value.get('examples', [None])[0]
            if example is not None:
                json_obj[key] = example


update_json_with_example(empty_json, schema_data)

# Print resulting JSON
#print(json.dumps(empty_json, indent=2))

validator = jsonschema.Draft7Validator(schema_data)
errors = list(validator.iter_errors(empty_json))

# Print detailed error messages for any validation errors
if errors:
    print(f"Validation failed with {len(errors)} error(s):")
    for i, error in enumerate(errors, start=1):
        print(f"Error {i}:")
        print(f"  Message: {error.message}")
        print(f"  Path: {'.'.join(str(p) for p in error.absolute_path)}")
        print(f"  Schema path: {'/'.join(str(p) for p in error.schema_path)}")
else:
    print("Validation successful!")

Validation failed with 2 error(s):
Error 1:
  Message: {} is not valid under any of the given schemas
  Path: dataDisclosed.0
  Schema path: properties/dataDisclosed/items/anyOf
Error 2:
  Message: {} is not valid under any of the given schemas
  Path: sources.0
  Schema path: properties/sources/items/anyOf


In [11]:
empty_json['dataDisclosed']

[{}]